# Questions

* Bcp de mots étranges révélés par le tf-idf, est ce normal ? mon regex tokenizer est il bon ?

<img src="14858512893477_so-logo.png" width="600px">    

<h1 style="font-family:verdana;font-size:300%;text-align:left;">Catégorisez automatiquement des questions</h1>

---
<a id="top"></a>
<h1 style="font-family:verdana;font-size:300%;text-align:left;">Sommaire</h1>


* [1. Sujet](#1)
    * [1.1 Contexte](#1.1)
    * [1.2 Extraction SQL](#1.2) 

In [11]:
%matplotlib inline

# Generic librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
from flask import Flask

# pd.set_option('display.max_column', 100)

# Activate multi output
InteractiveShell.ast_node_interactivity = "all"

# # For jupyter NB users 
# # set size of window
# display(HTML(data="""
# <style>
#     div#notebook-container    { width: 95%; }
#     div#menubar-container     { width: 65%; }
#     div#maintoolbar-container { width: 99%; }
# </style>
# """))

In [3]:
file = "data/data_clean.csv"
df = pd.read_csv(file, index_col='CreationDate', parse_dates=True)

In [4]:
df.head()
df.shape

,Score,ViewCount,Body,Title,Tags,AnswerCount,CommentCount,FavoriteCount,Body_tok,Title_tok,Tags_tok,Text_tok,Text
CreationDate,,,,,,,,,,,,,
2010-09-27 06:56:36,152,66955,i lately had the problem of creating add and e...,good or bad practice for dialogs in wpf with m...,<c#><.net><wpf><mvvm><modal-dialog>,3,12,115,"['lately', 'problem', 'creating', 'add', 'edit...","['good', 'bad', 'practice', 'dialog', 'wpf', '...","['c#', '.net', 'wpf', 'mvvm', 'modal-dialog']","['good', 'bad', 'practice', 'dialog', 'wpf', '...",good or bad practice for dialogs in wpf with m...
2008-08-11 08:55:13,147,253229,"in c, are the shift operators (<<,>>) arithmet...","are the shift operators (<<, >>) arithmetic or...",<c><binary><bit-manipulation><bit-shift>,11,1,60,"['c', 'shift', 'operator', 'arithmetic', 'logi...","['shift', 'operator', 'arithmetic', 'logical',...","['c', 'binary', 'bit-manipulation', 'bit-shift']","['shift', 'operator', 'arithmetic', 'logical',...","are the shift operators (<<, >>) arithmetic or..."
2008-08-11 10:27:22,815,576716,"whenever i design a database, i always wonder ...","database, table and column naming conventions?",<database><database-design><language-agnostic>...,23,4,359,"['whenever', 'design', 'database', 'always', '...","['database', 'table', 'column', 'naming', 'con...","['database', 'database-design', 'language-agno...","['database', 'table', 'column', 'naming', 'con...","database, table and column naming conventions?..."
2008-08-11 11:57:17,110,238131,is this even a valid question? i have a .net w...,how do i enable msdtc on sql server?,<sql-server><msdtc>,6,1,30,"['even', 'valid', 'question', 'net', 'window',...","['enable', 'msdtc', 'sql', 'server']","['sql-server', 'msdtc']","['enable', 'msdtc', 'sql', 'server', 'even', '...",how do i enable msdtc on sql server? is this e...
2008-08-10 18:41:02,539,197771,"what does the expression ""turing complete"" mea...",what is turing complete?,<theory><turing-machines><turing-complete>,15,1,178,"['expression', 'turing', 'complete', 'mean', '...","['turing', 'complete']","['theory', 'turing-machines', 'turing-complete']","['turing', 'complete', 'expression', 'turing',...",what is turing complete? what does the express...


(28077, 13)

Tf-idf

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
vectorizer = TfidfVectorizer(lowercase=False)
X = vectorizer.fit_transform(df['Text_tok'])

In [7]:
df['Text_tok']

CreationDate
2010-09-27 06:56:36    ['good', 'bad', 'practice', 'dialog', 'wpf', '...
2008-08-11 08:55:13    ['shift', 'operator', 'arithmetic', 'logical',...
2008-08-11 10:27:22    ['database', 'table', 'column', 'naming', 'con...
2008-08-11 11:57:17    ['enable', 'msdtc', 'sql', 'server', 'even', '...
2008-08-10 18:41:02    ['turing', 'complete', 'expression', 'turing',...
                                             ...                        
2017-01-29 10:58:46    ['multiprocessing', 'use', 'tqdm', 'display', ...
2017-01-30 03:16:39    ['cannot', 'appear', 'descendant', 'seeing', '...
2017-01-30 03:56:48    ['parse', 'json', 'kotlin', 'receiving', 'quit...
2017-01-30 14:47:03    ['download', 'file', 'using', 'axios', 'using'...
2018-04-26 23:29:11    ['virtualbox', 'raw', 'mode', 'unavailable', '...
Name: Text_tok, Length: 28077, dtype: object

In [8]:
# vectorizer.get_feature_names()
pd.DataFrame(X.todense(), columns=vectorizer.vocabulary_)

,good,bad,practice,dialog,wpf,mvvm,lately,problem,creating,add,...,implanted,validatedomnesting,someothercomponent,reacttooltip,ofsomecomponentandsomeothercomponent,reacttooltipfromreact,ofsomeothercomponent,rendersthis,supdrv,consolewrap
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Split

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df['Text_tok'], df['Tags_tok'], test_size=0.3, random_state=42)